In [2]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import numpy as np
from collections import defaultdict

<br>
<br>
<br>
<br>
<br>

## 3. Downloading the data

In [6]:
%cd /media/dan/data/School/Block_6/DSCI525/DSCI_525_Group_007
## Change it to the location that you want to download your files to.

/media/dan/data/School/Block_6/DSCI525/DSCI_525_Group_007


In [4]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshareairline/"

In [7]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [8]:
%%time
files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 3.56 s, sys: 7.33 s, total: 10.9 s
Wall time: 2min 18s


In [3]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

NameError: name 'output_directory' is not defined

In [7]:
%ls -ltr figshareairline/

total 5764496
-rwxrwxrwx 1 dan dan  95376895 Mar 27 14:38 MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv*
-rwxrwxrwx 1 dan dan  94960113 Mar 27 14:38 AWI-ESM-1-1-LR_daily_rainfall_NSW.csv*
-rwxrwxrwx 1 dan dan  82474546 Mar 27 14:38 NorESM2-LM_daily_rainfall_NSW.csv*
-rwxrwxrwx 1 dan dan 127613760 Mar 27 14:38 ACCESS-CM2_daily_rainfall_NSW.csv*
-rwxrwxrwx 1 dan dan 232118894 Mar 27 14:38 FGOALS-f3-L_daily_rainfall_NSW.csv*
-rwxrwxrwx 1 dan dan 330360682 Mar 27 14:38 CMCC-CM2-HR4_daily_rainfall_NSW.csv*
-rwxrwxrwx 1 dan dan 254009247 Mar 27 14:38 MRI-ESM2-0_daily_rainfall_NSW.csv*
-rwxrwxrwx 1 dan dan 235661418 Mar 27 14:38 GFDL-CM4_daily_rainfall_NSW.csv*
-rwxrwxrwx 1 dan dan 294260911 Mar 27 14:38 BCC-CSM2-MR_daily_rainfall_NSW.csv*
-rwxrwxrwx 1 dan dan 295768615 Mar 27 14:38 EC-Earth3-Veg-LR_daily_rainfall_NSW.csv*
-rwxrwxrwx 1 dan dan 328852379 Mar 27 14:38 CMCC-ESM2_daily_rainfall_NSW.csv*
-rwxrwxrwx 1 dan dan  67784105 Mar 27 14:38 NESM3_daily_rainfall_NSW.csv*
-rwxrwxrwx 1 dan dan  95640

<br>
<br>
<br>
<br>
<br>

## 4. Loading the data with Pandas

In [8]:
%%time
## This is all the data as is

files = glob.glob('figshareairline/*.csv')
files.remove("figshareairline/observed_daily_rainfall_SYD.csv")
df = pd.concat((pd.read_csv(file, index_col=0)
                .assign(model=re.findall("/([^_]*)", file)[0])
                for file in files)
              )
df.to_csv("figshareairline/combined_data.csv")

CPU times: user 8min 8s, sys: 30.1 s, total: 8min 38s
Wall time: 10min 41s


| Team Member |	Operating System | RAM | Processor | Is SSD  | Time taken 
|---|---|---|---|---|---|
| Alex  |  MacOS | 24gb  | M2  | Yes  | 3min 14s  |
| Daniel  |Ubuntu 22.04.2 LTS|20 GB|Intel® Core™ i5-6200U CPU @ 2.30GHz × 4|Yes|10min 41s|
| Ranjit  |       |   |   |   |   |
| Gaoxiang |      |   |   |   |   |



<br>
<br>
<br>
<br>
<br>

## 5. Python EDA

In [13]:
%%time

df.describe()

CPU times: user 11.6 s, sys: 13.1 s, total: 24.7 s
Wall time: 24.8 s


,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
count,5.924854e+07,6.246784e+07,5.924854e+07,6.246784e+07,5.924854e+07
mean,-3.310482e+01,-3.197757e+01,1.469059e+02,1.482150e+02,1.901170e+00
std,1.963549e+00,1.992067e+00,3.793784e+00,3.809994e+00,5.585735e+00
min,-3.646739e+01,-3.600000e+01,1.406250e+02,1.412500e+02,-3.807373e-12
25%,-3.486911e+01,-3.366221e+01,1.434375e+02,1.450000e+02,3.838413e-06
50%,-3.300000e+01,-3.204188e+01,1.468750e+02,1.481250e+02,6.154947e-02
75%,-3.140170e+01,-3.015707e+01,1.501875e+02,1.513125e+02,1.020918e+00
max,-2.990000e+01,-2.790606e+01,1.537500e+02,1.556250e+02,4.329395e+02


In [14]:
%%time

df[['lat_min','lat_max','lon_min','lon_max','rain (mm/day)']].astype('float32', errors='ignore').describe()

CPU times: user 9.56 s, sys: 3.79 s, total: 13.3 s
Wall time: 13.4 s


,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
count,5.924854e+07,6.246784e+07,5.924854e+07,6.246784e+07,5.924854e+07
mean,-3.310468e+01,-3.197750e+01,1.469054e+02,1.482152e+02,1.901175e+00
std,1.963549e+00,1.992067e+00,3.793784e+00,3.809994e+00,5.585735e+00
min,-3.646739e+01,-3.600000e+01,1.406250e+02,1.412500e+02,-3.807373e-12
25%,-3.486911e+01,-3.366221e+01,1.434375e+02,1.450000e+02,3.838413e-06
50%,-3.300000e+01,-3.204189e+01,1.468750e+02,1.481250e+02,6.154947e-02
75%,-3.140170e+01,-3.015707e+01,1.501875e+02,1.513125e+02,1.020918e+00
max,-2.990000e+01,-2.790606e+01,1.537500e+02,1.556250e+02,4.329395e+02


In [15]:
%%time

df.loc[:,'rain (mm/day)'].describe()

CPU times: user 3.07 s, sys: 132 ms, total: 3.2 s
Wall time: 3.2 s


count    5.924854e+07
mean     1.901170e+00
std      5.585735e+00
min     -3.807373e-12
25%      3.838413e-06
50%      6.154947e-02
75%      1.020918e+00
max      4.329395e+02
Name: rain (mm/day), dtype: float64

| Team Member |	Operating System | RAM | Processor | Is SSD  | Time taken (total; casted; single column)
|---|---|---|---|---|---|
| Alex  |  MacOS | 24gb  | M2  | Yes  | 8.62 s; 7.81 s; 2.2 s  |
| Daniel  |Ubuntu 22.04.2 LTS|20 GB|Intel® Core™ i5-6200U CPU @ 2.30GHz × 4|Yes|24.8 s; 13.4 s; 3.2 s|
| Ranjit  |       |   |   |   |   |
| Gaoxiang |      |   |   |   |   |


The fastest approach that we tested was to use only the column that we wanted to summarize (which in this case was the rain). Casting the data as float 32s did improve speed but the difference was not dramatic. 

<br>
<br>
<br>
<br>
<br>

## 6. R EDA

In [58]:
%load_ext rpy2.ipython


In [59]:
filepathcsv = "/Users/ataciuk/Documents/MDS/525_cloud-computing/figshareairline/combined_data.csv"

In [67]:
import pyarrow.dataset as ds
import pyarrow as pa
from pyarrow import csv
import rpy2_arrow.pyarrow_rarrow as pyra

In [68]:
%%time

dataset = ds.dataset(filepathcsv, format="csv")
# Converting the `pyarrow dataset` to a `pyarrow table`
table = dataset.to_table()
# Converting a `pyarrow table` to a `rarrow table`
r_table = pyra.converter.py2rpy(table)

CPU times: user 11 s, sys: 1.21 s, total: 12.3 s
Wall time: 11.4 s


In [71]:
%%time

%%R -i r_table

suppressMessages(library(dplyr))
result <- r_table %>% count(model)
print(result %>% collect())


# A tibble: 27 × 2
   model                  n
   <chr>              <int>
 1 MPI-ESM-1-2-HAM   966420
 2 AWI-ESM-1-1-LR    966420
 3 NorESM2-LM        919800
 4 ACCESS-CM2       1932840
 5 FGOALS-f3-L      3219300
 6 CMCC-CM2-HR4     3541230
 7 MRI-ESM2-0       3037320
 8 GFDL-CM4         3219300
 9 BCC-CSM2-MR      3035340
10 EC-Earth3-Veg-LR 3037320
# ℹ 17 more rows
# ℹ Use `print(n = ...)` to see more rows
CPU times: user 1.17 s, sys: 282 ms, total: 1.46 s
Wall time: 260 ms


In [72]:
%%time

%%R -i r_table

suppressMessages(library(dplyr))
result <- r_table %>% count(model)
print(result %>% collect())


# A tibble: 62,467,843 × 7
   time                lat_min lat_max lon_min lon_max `rain (mm/day)` model    
   <dttm>                <dbl>   <dbl>   <dbl>   <dbl>           <dbl> <chr>    
 1 1889-01-01 04:00:00   -35.4   -33.6    142.    143.        4.24e-13 MPI-ESM-…
 2 1889-01-02 04:00:00   -35.4   -33.6    142.    143.        4.22e-13 MPI-ESM-…
 3 1889-01-03 04:00:00   -35.4   -33.6    142.    143.        4.50e-13 MPI-ESM-…
 4 1889-01-04 04:00:00   -35.4   -33.6    142.    143.        4.25e-13 MPI-ESM-…
 5 1889-01-05 04:00:00   -35.4   -33.6    142.    143.        4.27e-13 MPI-ESM-…
 6 1889-01-06 04:00:00   -35.4   -33.6    142.    143.        4.20e-13 MPI-ESM-…
 7 1889-01-07 04:00:00   -35.4   -33.6    142.    143.        4.19e-13 MPI-ESM-…
 8 1889-01-08 04:00:00   -35.4   -33.6    142.    143.        4.56e-13 MPI-ESM-…
 9 1889-01-09 04:00:00   -35.4   -33.6    142.    143.        2.53e+ 0 MPI-ESM-…
10 1889-01-10 04:00:00   -35.4   -33.6    142.    143.        4.12e- 2 MPI-ESM-…
#

In [69]:
%%R -i r_table

result

Table (query)
model: string
n: int32

See $.data for the source Arrow object
